In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import spacy
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoCV, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

### Data Cleaning

In [2]:
# data cleaning

# read data
d = pd.read_csv('df_eng.csv', index_col = 0)
product = list(d['phone_url'].str.split('/'))


# parse brand and model from phont_url and add back to the dataframe
product2 = []
for i in product:
    product2.append(i[2])
    
product2_split = []
for p in product2:
    product2_split.append(p.split('-'))

brands = []
for p in product2_split:
    brands.append(p[0])
    
d['brand'] = brands

models = []
for p in product2_split:
    models.append(p[1])
    
d['models'] = models


# delete unused column
del d['lang']
del d['country']
del d['domain']
del d['score_max']
del d['author']
del d['product']


# drop columns with nan score
d.dropna(subset=['score'], inplace = True)
d.dropna(subset = ['extract'], inplace = True)

# lowercase extract text
d['extract'] = d.apply(lambda row: str(row['extract']).lower(),axis=1)

In [3]:
# d.to_csv('data_cleaned.csv')
d.head()

,phone_url,date,source,score,extract,brand,models
0,/cellphones/samsung-galaxy-s8/,5/2/2017,Verizon Wireless,10.0,as a diehard samsung fan who has had every sam...,samsung,galaxy
1,/cellphones/samsung-galaxy-s8/,4/28/2017,Phone Arena,10.0,love the phone. the phone is sleek and smooth ...,samsung,galaxy
2,/cellphones/samsung-galaxy-s8/,5/4/2017,Amazon,6.0,adequate feel. nice heft. processor's still sl...,samsung,galaxy
3,/cellphones/samsung-galaxy-s8/,5/2/2017,Samsung,9.2,never disappointed. one of the reasons i've be...,samsung,galaxy
4,/cellphones/samsung-galaxy-s8/,5/11/2017,Verizon Wireless,4.0,i've now found that i'm in a group of people t...,samsung,galaxy


In [14]:
# sample 100,000 data
random.seed(1)
d_sample = d.sample(100000)

### Tokenization and Lemmatization

In [16]:
# tokenization and lemmatization function
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import string
lemmatizer = WordNetLemmatizer()

# make my own function that takes in a full sentence, tokenizes it, lemmatizes the words, then joins it back on white space
# ref: taken from HW2 insturctor solution
def lemmatize_sentence(sentence):
    words = word_tokenize(sentence)
    res_words = []
    for word in words:
        res_words.append(lemmatizer.lemmatize(word).strip(string.punctuation))
    return " ".join(res_words)

In [17]:
d['extract_str'] = d.apply(lambda row: str(row['extract']),axis = 1)

In [18]:
d.head()

,phone_url,date,source,score,extract,brand,models,extract_str
0,/cellphones/samsung-galaxy-s8/,5/2/2017,Verizon Wireless,10.0,as a diehard samsung fan who has had every sam...,samsung,galaxy,as a diehard samsung fan who has had every sam...
1,/cellphones/samsung-galaxy-s8/,4/28/2017,Phone Arena,10.0,love the phone. the phone is sleek and smooth ...,samsung,galaxy,love the phone. the phone is sleek and smooth ...
2,/cellphones/samsung-galaxy-s8/,5/4/2017,Amazon,6.0,adequate feel. nice heft. processor's still sl...,samsung,galaxy,adequate feel. nice heft. processor's still sl...
3,/cellphones/samsung-galaxy-s8/,5/2/2017,Samsung,9.2,never disappointed. one of the reasons i've be...,samsung,galaxy,never disappointed. one of the reasons i've be...
4,/cellphones/samsung-galaxy-s8/,5/11/2017,Verizon Wireless,4.0,i've now found that i'm in a group of people t...,samsung,galaxy,i've now found that i'm in a group of people t...


In [21]:
# lemmatization
d['extract_lemma'] = d.apply(lambda row: lemmatize_sentence(row['extract_str']), axis = 1)

In [22]:
d.head()

,phone_url,date,source,score,extract,brand,models,extract_str,extract_lemma
0,/cellphones/samsung-galaxy-s8/,5/2/2017,Verizon Wireless,10.0,as a diehard samsung fan who has had every sam...,samsung,galaxy,as a diehard samsung fan who has had every sam...,a a diehard samsung fan who ha had every samsu...
1,/cellphones/samsung-galaxy-s8/,4/28/2017,Phone Arena,10.0,love the phone. the phone is sleek and smooth ...,samsung,galaxy,love the phone. the phone is sleek and smooth ...,love the phone the phone is sleek and smooth ...
2,/cellphones/samsung-galaxy-s8/,5/4/2017,Amazon,6.0,adequate feel. nice heft. processor's still sl...,samsung,galaxy,adequate feel. nice heft. processor's still sl...,adequate feel nice heft processor s still sl...
3,/cellphones/samsung-galaxy-s8/,5/2/2017,Samsung,9.2,never disappointed. one of the reasons i've be...,samsung,galaxy,never disappointed. one of the reasons i've be...,never disappointed one of the reason i ve bee...
4,/cellphones/samsung-galaxy-s8/,5/11/2017,Verizon Wireless,4.0,i've now found that i'm in a group of people t...,samsung,galaxy,i've now found that i'm in a group of people t...,i ve now found that i m in a group of people t...


### Vectorization

In [23]:
# vectorization
stopwords2 = stopwords.words() + ['phone', 'phones', 'cell', 'mobile'] # + brand ?

vectorizer = TfidfVectorizer(ngram_range = (2,4),
                             token_pattern = r'\b[a-zA-Z]{3,}\b', # detect text with three or more alphanumeric words
                             max_df = 0.4,
                             min_df = 2,
                             stop_words = stopwords2,
                             max_features = 200 # keep top 200 features
                            )

In [27]:
d.head()

,phone_url,date,source,score,extract,brand,models,extract_str,extract_lemma
0,/cellphones/samsung-galaxy-s8/,5/2/2017,Verizon Wireless,10.0,as a diehard samsung fan who has had every sam...,samsung,galaxy,as a diehard samsung fan who has had every sam...,a a diehard samsung fan who ha had every samsu...
1,/cellphones/samsung-galaxy-s8/,4/28/2017,Phone Arena,10.0,love the phone. the phone is sleek and smooth ...,samsung,galaxy,love the phone. the phone is sleek and smooth ...,love the phone the phone is sleek and smooth ...
2,/cellphones/samsung-galaxy-s8/,5/4/2017,Amazon,6.0,adequate feel. nice heft. processor's still sl...,samsung,galaxy,adequate feel. nice heft. processor's still sl...,adequate feel nice heft processor s still sl...
3,/cellphones/samsung-galaxy-s8/,5/2/2017,Samsung,9.2,never disappointed. one of the reasons i've be...,samsung,galaxy,never disappointed. one of the reasons i've be...,never disappointed one of the reason i ve bee...
4,/cellphones/samsung-galaxy-s8/,5/11/2017,Verizon Wireless,4.0,i've now found that i'm in a group of people t...,samsung,galaxy,i've now found that i'm in a group of people t...,i ve now found that i m in a group of people t...


In [37]:
d = d.reset_index(drop = True)

In [38]:
# vectorization
corpus = list(d['extract_lemma'].values)
X = vectorizer.fit_transform(corpus)
features = X.toarray()
d_formodel=pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())
d_formodel['TARGET'] = d['score']  # adding target to the data frame
d_formodel.index = d['extract'] # adding review to the data frame for future review

### Split Train and Test Sets

In [40]:
# feature selection and modeling - LASSO

# split train, test sets
train_df, test_df = train_test_split(d_formodel)

X_train = train_df.loc[:, ~train_df.columns.isin(['TARGET'])] # remove target
X_test = test_df.loc[:, ~test_df.columns.isin(['TARGET'])]

y_train = train_df['TARGET'] # get target
y_test = test_df['TARGET']

### Baseline (mean)

In [41]:
# baseline (mean)
np.mean(d_sample['score'])

7.686609999999868

In [42]:
# baseline train RMSE
baseline_rmse_train = pd.DataFrame(train_df['TARGET'])
baseline_rmse_train['predict'] = np.mean(d_sample['score'])
baseline_rmse_train['rmse'] = (baseline_rmse_train['TARGET'] - baseline_rmse_train['predict']) ** 2
(sum(baseline_rmse_train['rmse'])/ len(baseline_rmse_train)) ** 1/2   #RMSE

4.167074919068554

In [44]:
# train set R2
round(r2_score(y_true = baseline_rmse_train['TARGET'], y_pred = baseline_rmse_train['predict']), 2)

-0.0

In [46]:
# baseline test RMSE
baseline_rmse_test = pd.DataFrame(test_df['TARGET'])
baseline_rmse_test['predict'] = np.mean(d_sample['score'])
baseline_rmse_test['rmse'] = (baseline_rmse_test['TARGET'] - baseline_rmse_test['predict']) ** 2
(sum(baseline_rmse_test['rmse'])/ len(baseline_rmse_test)) ** 1/2   #RMSE

4.161516164824777

In [47]:
# train set R2
round(r2_score(y_true = baseline_rmse_test['TARGET'], y_pred = baseline_rmse_test['predict']), 2)

-0.0

### LASSO Regression with alpha = 0.005

In [48]:
# LASSO model with alpha = 0.005
# fit model to train
reg = Lasso(alpha = 0.005) 
reg.fit(X_train, y_train)
coef_lasso = pd.Series(reg.coef_, index = X_train.columns)
print("Lasso picked " + str(sum(coef_lasso != 0)) + " variables and eliminated the other " +  str(sum(coef_lasso == 0)) + " variables")

Lasso picked 30 variables and eliminated the other 170 variables


In [49]:
# predict for train set
y_pred = reg.predict(X_train)

lasso_rmse_train = pd.DataFrame(train_df['TARGET'])
lasso_rmse_train['predict'] = y_pred
lasso_rmse_train['rmse'] = (lasso_rmse_train['TARGET'] - lasso_rmse_train['predict']) ** 2
(sum(lasso_rmse_train['rmse'])/ len(lasso_rmse_train)) ** 1/2   #RMSE

3.8547809058075946

In [51]:
# train set R2
r2_score(y_true = lasso_rmse_train['TARGET'], y_pred = lasso_rmse_train['predict'])

0.07494222595829647

In [52]:
# predict for test set
y_pred = reg.predict(X_test)

lasso_rmse_test = pd.DataFrame(test_df['TARGET'])
lasso_rmse_test['predict'] = y_pred
lasso_rmse_test['rmse'] = (lasso_rmse_test['TARGET'] - lasso_rmse_test['predict']) ** 2
(sum(lasso_rmse_test['rmse'])/ len(lasso_rmse_test)) ** 1/2   #RMSE

3.844003060576442

In [53]:
# test set R2
r2_score(y_true = lasso_rmse_test['TARGET'], y_pred = lasso_rmse_test['predict'])

0.07629195198275218

### LASSO Regression Cross Validation Model

In [54]:
# LASSO CV model
# fit model to train
reg_cv = LassoCV()
reg_cv.fit(X_train, y_train)
coef_regcv = pd.Series(reg_cv.coef_, index = X_train.columns)
print("Lasso picked " + str(sum(coef_regcv != 0)) + " variables and eliminated the other " +  str(sum(coef_regcv == 0)) + " variables")

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Lasso picked 197 variables and eliminated the other 3 variables


In [55]:
# predict for train set
y_pred = reg_cv.predict(X_train)

lassocv_rmse_train = pd.DataFrame(train_df['TARGET'])
lassocv_rmse_train['predict'] = y_pred
lassocv_rmse_train['rmse'] = (lassocv_rmse_train['TARGET'] - lassocv_rmse_train['predict']) ** 2
(sum(lassocv_rmse_train['rmse'])/ len(lassocv_rmse_train)) ** 1/2   # RMSE

3.5904381447044953

In [56]:
# train set R2
r2_score(y_true = lassocv_rmse_train['TARGET'], y_pred = lassocv_rmse_train['predict'])

0.13837834130742577

In [57]:
# predict for test set
y_pred = reg_cv.predict(X_test)

lassocv_rmse_test = pd.DataFrame(test_df['TARGET'])
lassocv_rmse_test['predict'] = y_pred
lassocv_rmse_test['rmse'] = (lassocv_rmse_test['TARGET'] - lassocv_rmse_test['predict']) ** 2
(sum(lassocv_rmse_test['rmse'])/ len(lassocv_rmse_test)) ** 1/2   # RMSE

3.573596055865725

In [58]:
# train set R2
r2_score(y_true = lassocv_rmse_test['TARGET'], y_pred = lassocv_rmse_test['predict'])

0.14127034106081493

### Coefficients of Best Model (LASSO CV)

In [59]:
# most negative mentioned
coef_regcv_desc = coef_regcv.sort_values(ascending = False)
coef_regcv_desc[:20]

excellent product    2.402269
best ever            2.266377
best smartphone      2.255996
absolutely love      2.197169
love new             2.166301
work perfectly       2.123162
highly recommend     2.116530
love love            2.115676
great product        2.083842
far best             2.060095
super fast           1.966025
best price           1.936327
great love           1.936180
love everything      1.903395
easy set             1.870449
one best             1.867354
work great           1.829445
happy purchase       1.792538
really love          1.782445
doe everything       1.776124
dtype: float64

In [60]:
# most positive mentioned
coef_regcv_asc = coef_regcv.sort_values()
coef_regcv_asc[:20]

worst ever         -5.699143
waste money        -4.858022
dont buy           -4.600128
stopped working    -3.999490
doe work           -3.617142
working properly   -3.438906
hold charge        -2.488924
battery drain      -2.147692
every time         -2.118779
two month          -1.889037
one month          -1.867226
heating problem    -1.682193
battery doe        -1.596176
could get          -1.441465
make call          -1.396745
text message       -1.365618
many time          -1.330714
one day            -1.214200
sim card           -1.072702
two week           -1.034376
dtype: float64